In [1]:
%pip install python_dotenv
%pip install -U langchain langchain_experimental
%pip install -U spotipy
%pip install -U openai==v0.28.1
%pip install -U gradio==3.48.0
%pip install torch

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install pydantic

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install -qU \
    pinecone-client \
    pinecone-datasets \
    tiktoken

Note: you may need to restart the kernel to use updated packages.


In [4]:
%reload_ext dotenv
%dotenv

In [4]:
from langchain_experimental.pal_chain import PALChain
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.prompts.prompt import PromptTemplate

In [5]:
from spotipy.oauth2 import SpotifyOAuth, SpotifyClientCredentials

In [6]:
import os

In [7]:
llm = ChatOpenAI(model="gpt-3.5-turbo")

In [8]:
auth = SpotifyOAuth(
    client_id=os.getenv("SPOTIFY_CLIENT_ID"),
    client_secret=os.getenv("SPOTIFY_CLIENT_SECRET"),
    redirect_uri="http://localhost:8888/callback",
    scope="user-library-read user-read-playback-position user-read-recently-played user-top-read",
)

In [9]:
import spotipy
spotifyClient = spotipy.Spotify(auth_manager=auth)
print(spotifyClient.me())

{'display_name': 'Arun', 'external_urls': {'spotify': 'https://open.spotify.com/user/31ajryiecdu4qlhbplqxxtrcyvbu'}, 'href': 'https://api.spotify.com/v1/users/31ajryiecdu4qlhbplqxxtrcyvbu', 'id': '31ajryiecdu4qlhbplqxxtrcyvbu', 'images': [], 'type': 'user', 'uri': 'spotify:user:31ajryiecdu4qlhbplqxxtrcyvbu', 'followers': {'href': None, 'total': 0}}


In [ ]:
%pip install datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset("maharshipandya/spotify-tracks-dataset")

In [ ]:
print(dataset.column_names)

In [ ]:
import random

# Function to generate a sentence for danceability
def danceability_sentence(danceability):
    phrases = [
        f"With a danceability score of {danceability}, this track makes you want to get up and move.",
        f"The track has a danceability rating of {danceability}, perfect for dancing.",
        f"You'll find yourself swaying to the rhythm, thanks to its danceability of {danceability}."
    ]
    return random.choice(phrases)

# Function to generate a sentence for energy
def energy_sentence(energy):
    if energy > 0.7:
        return f"It's a high-energy track with a score of {energy}, radiating vibrant vibes."
    elif energy < 0.3:
        return f"With a calm energy level of {energy}, it's a more laid-back track."
    else:
        return f"The track has a balanced energy level of {energy}, neither too calm nor too intense."


In [ ]:

# Function for loudness
def loudness_sentence(loudness):
    if loudness > -5:
        return f"With a loudness of {loudness} dB, it's quite a loud track."
    elif loudness < -20:
        return f"It's a softer track with a loudness of {loudness} dB."
    else:
        return f"The track has a moderate loudness level of {loudness} dB."

# Function for mode
def mode_sentence(mode):
    mode_str = "major" if mode == 1 else "minor"
    return f"The track is in the {mode_str} mode."

# Function for speechiness
def speechiness_sentence(speechiness):
    if speechiness > 0.66:
        return "It's predominantly a spoken word track."
    elif 0.33 < speechiness <= 0.66:
        return "The track features a mix of music and speech."
    else:
        return "It's mostly music with minimal speech."

# Function for acousticness
def acousticness_sentence(acousticness):
    if acousticness > 0.5:
        return "This track has a high level of acoustic sound."
    else:
        return "It features more electronic than acoustic sounds."

# Function for instrumentalness
def instrumentalness_sentence(instrumentalness):
    if instrumentalness > 0.5:
        return "It's largely an instrumental track."
    else:
        return "The track includes vocal content."

# Function for liveness
def liveness_sentence(liveness):
    if liveness > 0.8:
        return "There's a high probability the track was recorded live."
    else:
        return "The track likely wasn't recorded in front of a live audience."

# Function for valence
def valence_sentence(valence):
    if valence > 0.5:
        return "It's a track with a more positive, happy vibe."
    else:
        return "The track has a more somber or melancholic tone."

# Function for tempo
def tempo_sentence(tempo):
    return f"The tempo of the track is around {tempo} BPM."

# Function for time signature
def time_signature_sentence(time_signature):
    return f"It has a {time_signature}/4 time signature."

# Incorporating these functions into create_description function
def create_description(example):
    parts = [
        f"Track '{example['track_name']}' by {example['artists']} from the album '{example['album_name']}'",
        danceability_sentence(example['danceability']),
        energy_sentence(example['energy']),
        loudness_sentence(example['loudness']),
        mode_sentence(example['mode']),
        speechiness_sentence(example['speechiness']),
        acousticness_sentence(example['acousticness']),
        instrumentalness_sentence(example['instrumentalness']),
        liveness_sentence(example['liveness']),
        valence_sentence(example['valence']),
        tempo_sentence(example['tempo']),
        time_signature_sentence(example['time_signature']),
        f"The genre is categorized as {example['track_genre']}.",
        f"It has a popularity score of {example['popularity']} and lasts for {example['duration_ms'] / 1000} seconds."
    ]
    example['textual_description'] = ' '.join(parts)
    return example


In [ ]:
dataset = dataset.map(create_description)

In [ ]:
print(dataset.column_names)
print(dataset["train"][0])

In [ ]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import pinecone
import numpy as np

small_dataset_to_vectorize = dataset['train'].select(range(10000))

# Initialize the SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Pinecone setup
pinecone.init(api_key='ef418cba-180e-498f-be6a-4c38711964d2', environment='gcp-starter')
index_name = 'llm-music-chatbot'

# Create the Pinecone index if it does not exist
if index_name not in pinecone.list_indexes():
    pinecone.create_index(index_name, dimension=384)

# Connect to the Pinecone index
index = pinecone.Index(index_name)

# Function to vectorize and upload to Pinecone
def vectorize_and_upload(batch):
    # Vectorize the textual descriptions in the batch
    embeddings = model.encode(batch['textual_description'])
    
    # Convert embeddings to lists
    embeddings_list = embeddings.tolist()

    # Create a list of tuples for uploading
    batch_for_upload = []
    for track_id, emb in zip(batch['track_id'], embeddings_list):
        # Ensure track_id is a string and add it to the batch
        batch_for_upload.append((str(track_id), emb))

    # Upload the batch to Pinecone
    index.upsert(vectors=batch_for_upload)

# Apply the function to the dataset
dataset.map(vectorize_and_upload, batched=True, batch_size=1000)

# from math import ceil

# total_rows = 114000
# chunk_size = 10000  # Process 10k rows at a time
# batch_size = 160    # Batch size for upsert

# # Calculate the number of chunks
# num_chunks = ceil(total_rows / chunk_size)

# for chunk in range(num_chunks):
#     start_row = chunk * chunk_size
#     end_row = min((chunk + 1) * chunk_size, total_rows)

#     # Slice the dataset for the current chunk
#     current_chunk = dataset["train"].select(range(start_row, end_row))

#     # Process and upsert the current chunk in batches
#     current_chunk.map(vectorize_and_upload, batched=True, batch_size=batch_size)

# Close the index connection when done
# index.close()  # Uncomment to close the Pinecone index connection


In [ ]:
def query_pinecone(query):
    query_vector = model.encode(query).tolist()
    results = index.query(vector=query_vector, top_k=5)
    return results

In [ ]:
query_pinecone("I want to listen to some beatles music that's rock")

In [ ]:
recently_played = spotifyClient.current_user_recently_played(limit=2)
print(recently_played)
for track in recently_played['items']:
    print(track['track']['name'], "-", track['track']['artists'][0]['name'])

In [ ]:
# Top Tracks
top_tracks = spotifyClient.current_user_top_tracks(limit=2)
print(top_tracks)
for track in top_tracks['items']:
    print(track['name'], "-", track['artists'][0]['name'])

# Top Artists
top_artists = spotifyClient.current_user_top_artists(limit=2)
print(top_artists)
for artist in top_artists['items']:
    print(artist['name'])

In [ ]:
saved_tracks = spotifyClient.current_user_saved_tracks(limit=2)
print(saved_tracks)
for item in saved_tracks['items']:
    track = item['track']
    print(track['name'], "-", track['artists'][0]['name'])

In [ ]:
playlists = spotifyClient.current_user_playlists(limit=2)
print(playlists)
for playlist in playlists['items']:
    print(playlist['name'])

In [ ]:
track_ids = [track['track']['id'] for track in recently_played['items']]

# Fetching audio features for the tracks
audio_features = spotifyClient.audio_features(track_ids)
print(audio_features)

for feature in audio_features:
    if feature:  # Check if feature data is available
        print(f"Audio Features for Track ID: {feature['id']}")
        for key, value in feature.items():
            print(f" - {key.capitalize().replace('_', ' ')}: {value}")
        print()

In [ ]:
# Fetch the top 10 artists
top_artists = spotifyClient.current_user_top_artists(limit=10)
for artist in top_artists['items']:
    artist_details = spotifyClient.artist(artist['id'])
    print(f"Artist: {artist_details['name']}")
    print(f"Genres: {', '.join(artist_details['genres'])}")
    print(f"Popularity: {artist_details['popularity']}")
    print()

# Fetch the top 10 tracks
top_tracks = spotifyClient.current_user_top_tracks(limit=10)
album_ids = set()  # To avoid duplicate albums
for track in top_tracks['items']:
    album_id = track['album']['id']
    if album_id not in album_ids:
        album_ids.add(album_id)
        album_details = spotifyClient.album(album_id)
        print(f"Album: {album_details['name']}")
        print(f"Release Date: {album_details['release_date']}")
        print(f"Total Tracks: {album_details['total_tracks']}")
        print()

In [ ]:
import pinecone
from sentence_transformers import SentenceTransformer
import json

In [ ]:
pinecone.init(api_key="ef418cba-180e-498f-be6a-4c38711964d2", environment='gcp-starter')
index_name = "llm-music-chatbot"
if index_name not in pinecone.list_indexes():
    pinecone.create_index(index_name, dimension=384, metric="cosine")  # Adjust dimension as needed
index = pinecone.Index(index_name)

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
# Example of vectorizing a piece of text
def vectorize(text):
    return model.encode(text).tolist()

In [ ]:
# Example of adding a vector to Pinecone
def upload_to_pinecone(data_strings, category):
    for i, data_string in enumerate(data_strings):
        vector = vectorize(data_string)
        id = f"{category}-{i}"
        index.upsert(vectors={id: vector})

In [ ]:
def query_pinecone(query):
    query_vector = vectorize(query)
    results = index.query(vector=query_vector, top_k=5)
    return results

In [ ]:
recently_played_strings = []
recently_played_track_ids = []
for item in recently_played['items']:
    track = item['track']
    track_string = f"Recently played track: {track['name']} by {track['artists'][0]['name']} from the album {track['album']['name']} on {item['played_at']}"
    recently_played_strings.append(track_string)
    recently_played_track_ids.append(track['id'])  # Add the track ID to the list
print(recently_played_strings)
print(recently_played_track_ids)

In [ ]:
top_tracks_strings = []
top_tracks_track_ids = []
for track in top_tracks['items']:
    track_string = f"Top track: {track['name']} by {track['artists'][0]['name']} from the album {track['album']['name']}"
    top_tracks_strings.append(track_string)
    top_tracks_track_ids.append(track['id'])  # Add the track ID to the list
print(top_tracks_strings)
print(top_tracks_track_ids)

In [ ]:
saved_tracks_strings = []
saved_tracks_track_ids = []
for item in saved_tracks['items']:
    track = item['track']
    track_string = f"Saved track: {track['name']} by {track['artists'][0]['name']} from the album {track['album']['name']}"
    saved_tracks_strings.append(track_string)
    saved_tracks_track_ids.append(track['id'])  # Add the track ID to the list
print(saved_tracks_strings)
print(saved_tracks_track_ids)

In [ ]:
top_artists_strings = []
top_artists_ids = []
for artist in top_artists['items']:
    artist_string = f"Top artist: {artist['name']} known for {', '.join(artist['genres'])} with popularity score of {artist['popularity']}"
    top_artists_strings.append(artist_string)
    top_artists_ids.append(artist['id'])  # Add the artist ID to the list
print(top_artists_strings)
print(top_artists_ids)

In [ ]:
audio_features_strings = []
audio_ids = []
for feature in audio_features:
    features_list = [f"{key}: {value}" for key, value in feature.items() if key != 'type' and key != 'uri' and key != 'track_href' and key != 'analysis_url']
    feature_string = f"Audio features for Track ID {feature['id']}: {', '.join(features_list)}"
    audio_features_strings.append(feature_string)
    audio_ids.append(feature['id'])  # Add the track ID to the list
print(audio_features_strings)
print(audio_ids)

In [ ]:
playlists_strings = []
playlists_ids = []
for playlist in playlists['items']:
    playlist_string = f"Playlist: {playlist['name']}, Description: {playlist['description']}, Total Tracks: {playlist['tracks']['total']}"
    playlists_strings.append(playlist_string)
    playlists_ids.append(playlist['id'])  # Add the playlist ID to the list
print(playlists_strings)
print(playlists_ids)

In [ ]:
# Modified function to work with a list of pre-formatted strings and a separate list of IDs
def upload_tracks_to_pinecone(track_strings, track_ids, category):
    batch = []
    for track_string, track_id in zip(track_strings, track_ids):
        vector = vectorize(track_string)
        pinecone_id = f"{category}-{track_id}"  # Create a unique identifier
        batch.append((pinecone_id, vector))
    
    # Upsert in batch
    index.upsert(vectors=batch)

# Example usage
upload_tracks_to_pinecone(recently_played_strings, recently_played_track_ids, 'recently_played')


In [ ]:
# upload top tracks to pinecone
upload_tracks_to_pinecone(top_tracks_strings, top_tracks_track_ids, 'top_tracks')


In [ ]:
upload_tracks_to_pinecone(saved_tracks_strings, saved_tracks_track_ids, 'saved_tracks')


In [ ]:
upload_tracks_to_pinecone(audio_features_strings, audio_ids, 'audio_features')  # Reusing top_tracks_ids for simplicity


In [ ]:
def upload_artists_to_pinecone(artist_strings, artist_ids, category):
    batch = []
    for artist_string, artist_id in zip(artist_strings, artist_ids):
        vector = vectorize(artist_string)
        pinecone_id = f"{category}-{artist_id}"  # Create a unique identifier
        batch.append((pinecone_id, vector))
    
    # Upsert in batch
    index.upsert(vectors=batch)

# Example usage (assuming you have top_artists_strings and top_artists_ids lists ready)
upload_artists_to_pinecone(top_artists_strings, top_artists_ids, 'top_artists')


In [ ]:
def upload_playlists_to_pinecone(playlist_strings, playlist_ids, category):
    batch = []
    for playlist_string, playlist_id in zip(playlist_strings, playlist_ids):
        vector = vectorize(playlist_string)
        pinecone_id = f"{category}-{playlist_id}"  # Create a unique identifier
        batch.append((pinecone_id, vector))
    
    # Upsert in batch
    index.upsert(vectors=batch)

# Example usage (assuming you have playlists_strings and playlists_ids lists ready)
upload_playlists_to_pinecone(playlists_strings, playlists_ids, 'playlists')


In [ ]:
query_pinecone("can you recommend some songs with high beat for gym workout?")

In [11]:
SPOTIPY_PROMPT_TEMPLATE = (
    '''
API LIMITATIONS TO NOTE
* When requesting track information, the limit is 50 at a time
* When requesting audio features, the limit is 100 at a time
* When selecting multiple artists, the limit is 50 at a time
* When asking for recommendations, the limit is 50 at a time
=====

Q: What albums has the band Linkin Park made?

# solution in Python:


def solution():
    """What albums has the band Linkin Park made?"""
    search_results = sp.search(q='Linkin Park', type='artist')
    uri = search_results['artists']['items'][0]['uri']
    albums = sp.artist_albums(green_day_uri, album_type='album')
    return albums




Q: Who are some musicians similar to Drake?

# solution in Python:


def solution():
    """Who are some musicians similar to Drake?"""
    search_results = sp.search(q='Drake', type='artist')
    uri = search_results['artists']['items'][0].get('uri')
    artists = sp.artist_related_artists(uri)
    return artists



Q: Tell me what songs by Anirudh sound like

# solution in Python:


def solution():
    """Tell me what songs by Anirudh sound like?"""
    search_results = sp.search(q='Anirudh', type='artist')
    uri = search_results['artists']['items'][0].get('uri')
    tracks = sp.artist_top_tracks(uri)
    track_uris = [track.get('uri') for track in tracks['tracks']]
    audio_details = sp.audio_features(track_uris)
    return audio_details



Q: Get me the URI for the album Living Things

# solution in Python:


def solution():
    """Get me the URI for the album Living Things"""
    search_results = sp.search(q='Living Things', type='album')
    uri = search_results['albums']['items'][0].get('uri')
    return uri



Q: What are the first three songs on Leo?

# solution in Python:


def solution():
    """What are the first three songs on Leo?"""
    # Get the URI for the album
    search_results = sp.search(q='Leo', type='album')
    album = search_results['albums']['items'][0]
    album_uri = album['uri']
    # Get the album tracks
    album_tracks = sp.album_tracks(album_uri)['items']
    # Sort the tracks by duration
    first_three = album_tracks[:3]
    tracks = []
    # Only include relevant fields
    for i, track in enumerate(first_three):
        # track['album'] does NOT work with sp.album_tracks
        # you need to use album['name'] instead
        tracks.append({{
            'position': i+1,
            'song_name': track.get('name'),
            'song_uri': track['artists'][0].get('uri'),
            'artist_uri': track['artists'][0].get('uri'),
            'album_uri': album.get('uri'),
            'album_name': album.get('name')
        }})
    return tracks


Q: What are the thirty most danceable songs by Rahman?

# solution in Python:


def solution():
    """What are most danceable songs by Rahman?"""
    search_results = sp.search(q='Rahman', type='artist')
    uri = search_results['artists']['items'][0]['uri']
    albums = sp.artist_albums(uri, album_type='album')
    album_uris = [album['uri'] for album in albums['items']]
    tracks = []
    for album_uri in album_uris:
        album_tracks = sp.album_tracks(album_uri)
        tracks.extend(album_tracks['items'])
    track_uris = [track['uri'] for track in tracks]
    danceable_tracks = []
    # You can only have 100 at a time
    for i in range(0, len(track_uris), 100):
        subset_track_uris = track_uris[i:i+100]
        audio_details = sp.audio_features(subset_track_uris)
        for j, details in enumerate(audio_details):
            if details['danceability'] > 0.7:
                track = tracks[i+j]
                danceable_tracks.append({{
                    'song': track.get('name')
                    'album': track.get('album').get('name')
                    'danceability': details.get('danceability'),
                    'tempo': details.get('tempo'),
                }})
                # Be sure to add the audio details to the track
                danceable_tracks.append(track)
    return danceable_tracks



Q: {question}. Return a list or dictionary, including all the fields, including relevant scores and the uris to the albums/songs/artists mentioned. Only return the data – if the prompt asks for a format such as markdown or a simple string, ignore it: you are only meant to provide the information, not the formatting. A later step in the process will convert the data into the new format (table, sentence, etc).

# solution in Python:
'''.strip()
    + "\n\n\n"
)

SPOTIPY_PROMPT = PromptTemplate(input_variables=["question"], template=SPOTIPY_PROMPT_TEMPLATE)

In [12]:
spotify_chain = PALChain(
    llm_chain=LLMChain(llm=ChatOpenAI(model="gpt-3.5-turbo"), prompt=SPOTIPY_PROMPT),
    get_answer_expr="import json; print(json.dumps(solution()))",
    python_globals={"sp": spotifyClient},
    stop="\n\n\n",
    verbose=True,
    return_intermediate_steps=True,
)
print(spotify_chain)

verbose=True llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='API LIMITATIONS TO NOTE\n* When requesting track information, the limit is 50 at a time\n* When requesting audio features, the limit is 100 at a time\n* When selecting multiple artists, the limit is 50 at a time\n* When asking for recommendations, the limit is 50 at a time\n=====\n\nQ: What albums has the band Green Day made?\n\n# solution in Python:\n\n\ndef solution():\n    """What albums has the band Green Day made?"""\n    search_results = sp.search(q=\'Green Day\', type=\'artist\')\n    uri = search_results[\'artists\'][\'items\'][0][\'uri\']\n    albums = sp.artist_albums(green_day_uri, album_type=\'album\')\n    return albums\n\n\n\n\nQ: Who are some musicians similar to Fiona Apple?\n\n# solution in Python:\n\n\ndef solution():\n    """Who are some musicians similar to Fiona Apple?"""\n    search_results = sp.search(q=\'Fiona Apple\', type=\'artist\')\n    uri = search_results[\'artist

In [13]:
PROMPT_TEMPLATE_TO_CLEANUP_RESPONSE = (""" 
Using this code:

```python
{intermediate_steps}
```

We got the following output from the Spotify API:

```json
{result}
```

Using the output above as your data source, answer the question {question}. Don't describe the code or process, just answer the question.
Answer:"""
)

PROMPT_RESPONSE_CLEANUP = PromptTemplate(
    input_variables=["question", "intermediate_steps", "result"],
    template=PROMPT_TEMPLATE_TO_CLEANUP_RESPONSE,
)

In [14]:
explainer_chain = LLMChain(
    llm=llm,
    prompt=PROMPT_RESPONSE_CLEANUP,
    verbose=True,
    output_key='answer'
)

In [15]:
overall_chain = SequentialChain(
    chains=[spotify_chain, explainer_chain],
    input_variables=['question'],
    verbose=True
)

In [22]:
print(overall_chain.run("I have listened to Linkin Park. What are some similar artists?"))



> Entering new SequentialChain chain...


> Entering new PALChain chain...
def solution():
    """What are some similar artists to Linkin Park?"""
    search_results = sp.search(q='Linkin Park', type='artist')
    uri = search_results['artists']['items'][0].get('uri')
    artists = sp.artist_related_artists(uri)
    related_artists = []
    for artist in artists['artists']:
        related_artists.append({
            'artist_name': artist.get('name'),
            'artist_uri': artist.get('uri')
        })
    return related_artists

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
 
Using this code:

```python
def solution():
    """What are some similar artists to Linkin Park?"""
    search_results = sp.search(q='Linkin Park', type='artist')
    uri = search_results['artists']['items'][0].get('uri')
    artists = sp.artist_related_artists(uri)
    related_artists = []
    for artist in artists['artists']:
        related_artists.append({
            'ar

In [23]:
import gradio as gr
import random
import time


/Users/arun/spotify-with-langchain/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
import gradio as gr
import random
import time

def respond(message, chat_history):
    # You can now use the name and age in your response logic if needed
    bot_message = overall_chain.run(message)
    chat_history.append((f"{message}", bot_message))
    time.sleep(2)
    return "", chat_history

with gr.Blocks() as demo:

    with gr.Row():
        chatbot = gr.Chatbot()
        msg = gr.Textbox(label="Your Message")
        clear = gr.ClearButton([msg, chatbot])

    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])

demo.launch(share=False)


Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.




> Entering new SequentialChain chain...


> Entering new PALChain chain...
def solution():
    """Give me some popular hip hop songs from Drake"""
    search_results = sp.search(q='Drake', type='artist')
    uri = search_results['artists']['items'][0]['uri']
    albums = sp.artist_albums(uri, album_type='album')
    album_uris = [album['uri'] for album in albums['items']]
    tracks = []
    for album_uri in album_uris:
        album_tracks = sp.album_tracks(album_uri)
        tracks.extend(album_tracks['items'])
    track_uris = [track['uri'] for track in tracks]
    popular_tracks = []
    # You can only have 50 at a time
    for i in range(0, len(track_uris), 50):
        subset_track_uris = track_uris[i:i+50]
        audio_details = sp.audio_features(subset_track_uris)
        for j, details in enumerate(audio_details):
            if details['popularity'] > 70:
                track = tracks[i+j]
                popular_tracks.append({
                    'song': track.get('name